# Knowledge Agents


## 1. Setup


In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Add these imports at the beginning of your notebook
import warnings
import logging
from IPython.display import clear_output

warnings.filterwarnings('ignore')

logging.getLogger().setLevel(logging.ERROR)
import nest_asyncio
nest_asyncio.apply()


In [2]:
from llama_index.core import (
    PromptTemplate,
    Settings,
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.response_synthesizers import ResponseMode

from llama_index.core.tools import QueryEngineTool, ToolMetadata
import sys
from loguru import logger

logger.add(sys.stdout, level="INFO", format="{time} {level} {message}", colorize=True)
logger.add("logs/nb_log_{time}.log", 
           rotation="10 MB",  # Rotate log file when it reaches 10MB
           retention="30 days",  # Keep logs for 30 days
           compression="zip",  # Optionally compress old logs to save space
           format="{time} {level} {message} {extra}")

2

## 2. Init the ML env


In [4]:
llm = Ollama(model="llama3.2:3b-instruct-q8_0", temperature=0.01, timeout=120)

In [ ]:
response = llm.complete("How are you?")

In [7]:
print(str(response))

I'm just a language model, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?


In [9]:
embedding = OllamaEmbedding(model_name="mxbai-embed-large")
Settings.llm = llm
Settings.embed_model = embedding

In [11]:
hello_embedding = embedding.get_text_embedding("hello")
type(hello_embedding), type(hello_embedding[0]), len(hello_embedding)

(list, float, 1024)

## 3. Load data


In [12]:
q1_docs = SimpleDirectoryReader(
        input_files=["../data/crm-2025-10Q-q1.pdf"]
    ).load_data()

In [13]:
q1_index = VectorStoreIndex.from_documents(
        q1_docs,
    )
q1_engine = q1_index.as_query_engine(
    similarity_top_k=5,
    response_mode=ResponseMode.TREE_SUMMARIZE,
)

## 4. Test Drive the RAG!


In [16]:
response = q1_engine.query("What is the total revenue for CRM in Q1?")
print(str(response))

The total revenue for CRM in Q1 is $9,133 million.
